In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', optimized=True, numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
np.set_printoptions(suppress=True, precision=3)

In [2]:
prefounders = sim.loadPopulation('prefounders1478.pop')

In [3]:
multi_prefounders = sim.Simulator(prefounders, 50, stealPops=False)

In [4]:
founders = [[1, 2], [3, 4], [5, 6], [7, 8]]
os_per_pair = 500

In [5]:
magic = breed.MAGIC(multi_prefounders, [0.01]*1478)

In [6]:
sim.tagID(prefounders, reset=27)

In [7]:
magic.generate_f_one(founders, os_per_pair)

Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0
Generation: 0


In [8]:
mrc = breed.MultiRandomCross(multi_prefounders, 4, 500)

In [9]:
mother_choices, father_choices = mrc.determine_random_cross()

In [10]:
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(mother_choices, father_choices)

In [11]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1)

In [12]:
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)

In [13]:
final_mothers, final_fathers = final_mrc.determine_random_cross()

In [14]:
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(final_mothers, final_fathers)

In [15]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1)

In [16]:
mater = breed.MAGIC(multi_prefounders, [0.01]*1478)

In [17]:
mater.random_mating(3, 2000)

Initiating random mating for 3 generations.
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 3
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Generation: 4
Gene

### Segregating Loci

In [18]:
for rep in multi_prefounders.populations():
    sim.stat(rep, numOfSegSites=sim.ALL_AVAIL, vars=['segSites', 'numOfSegSites'])

In [19]:
seg_loci_of_samples = np.zeros((50, 866, 3), dtype=np.int32)

In [21]:
sample_sizes = [100, 250, 500]

In [22]:
for rep in multi_prefounders.populations():
    for i, sample in enumerate(sample_sizes):
        sampled_pop = sim.sampling.drawRandomSample(rep, sample)
        sim.stat(sampled_pop, numOfSegSites=sim.ALL_AVAIL, vars=['segSites', 'numOfSegSites'])
        seg_loci_of_samples[rep.dvars().rep, :, i] = sampled_pop.dvars().segSites

### Determine QTL

In [23]:
number_qt_loci = 10

In [24]:
segregating_loci = list(seg_loci_of_samples[0, :, 0])

In [48]:
segregating_loci = [int(locus) for locus in segregating_loci]

In [25]:
qtl = sorted(random.sample(segregating_loci, number_qt_loci))

In [26]:
qtl = [int(locus) for locus in qtl]

numpy.int32

In [27]:
qtl

[85, 294, 378, 417, 431, 730, 935, 1108, 1348, 1355]

In [ ]:
def coefficient_of_dispersion(pop, loci):
    """
    Mean to variance ratio of pairwise sequential distances of quantitative trait loci.
    Note that this statistic contributes nothing if there is only one qtl on a chromosome.
    :param pop:
    """
    chrom_loc_pairs = [pop.chromLocusPair(int(locus)) for locus in loci]
    chromosomes = [chrom_loc_pairs[i][0] for i in range(len(chrom_loc_pairs))]
    diffs = []
    for i in range(len(chrom_loc_pairs)):
        if chromosomes[i-1] == chromosomes[i]:
            diffs.append(math.fabs(chrom_loc_pairs[i-1][1] - chrom_loc_pairs[i][1]))
    diffs = np.array(diffs)
    mean = np.mean(diffs)
    var = np.var(diffs)
    var_to_mean_ratio = var/mean
    return var_to_mean_ratio

In [ ]:
first = multi_prefounders.population(0)

In [ ]:
coefficient_of_dispersion(first, qtl)

In [28]:
alleles = pd.read_hdf('parameters\\alleles_at_1478_loci.hdf')

In [31]:
alleles = np.array(alleles)

In [32]:
add_trait = parameters.Trait()

In [33]:
allele_effects = add_trait.assign_allele_effects(alleles, qtl, random.expovariate, 1, multiplicity=3)

In [34]:
allele_effects

{85: {1: 0.7639459962395068, 3: 1.1275557092940487},
 294: {0: 0.8082841215038653, 2: 1.8820116489441723},
 378: {0: 7.048513796426754, 2: 1.4224519757549239},
 417: {1: 6.714168847163591, 3: 1.268012923400879},
 431: {1: 2.6270165938652026, 3: 4.909446892623217},
 730: {1: 3.378195420119303, 3: 3.752044147848409},
 935: {0: 3.1937192305039086, 2: 4.8342880250866},
 1108: {0: 3.214484353047612, 1: 5.40893005938693},
 1348: {1: 5.138900439370714, 3: 4.188077952052308},
 1355: {0: 3.323581565680311, 3: 5.605738561429297}}

## Save Population Samples

In [ ]:
sample_sizes = [100, 250, 500]

In [ ]:
for rep in multi_prefounders.populations():
    for sample in sample_sizes:
        sampled_pop = sim.sampling.drawRandomSample(rep, sample)
        sample_name = 'infinite_R_' + str(rep.dvars().rep) + ' _' + str(sampled_pop.popSize()) + '.pop'
        sampled_pop.save(sample_name)

In [53]:
import importlib as imp
imp.reload(analyze)

<module 'saegus.analyze' from 'C:\\Anaconda3\\lib\\site-packages\\saegus\\analyze.py'>

In [54]:
analyze.multiple_sample_analyzer(multi_prefounders, sample_sizes, qtl, alleles, allele_effects, 0.7, segregating_loci)

In [37]:
for rep in multi_prefounders.populations():
    print(rep.dvars().rep)
    break

0


In [55]:
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus

In [56]:
infinite_loci_conversions = shelve.open('infinite_loci_conversions')

In [57]:
infinite_loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
infinite_loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci

In [58]:
infinite_loci_conversions.close()

In [59]:
inf_ae_table = analyze.generate_allele_effects_table(qtl, alleles, allele_effects)

In [60]:
inf_ae_table

,locus,alpha_allele,alpha_effect,beta_allele,beta_effect
0,85,3,1.127556,1,0.763946
1,294,0,0.808284,2,1.882012
2,378,2,1.422452,0,7.048514
3,417,3,1.268013,1,6.714169
4,431,3,4.909447,1,2.627017
5,730,3,3.752044,1,3.378195
6,935,2,4.834288,0,3.193719
7,1108,0,3.214484,1,5.408930
8,1348,3,4.188078,1,5.138900
9,1355,3,5.605739,0,3.323582


In [61]:
expanded_inf_ae = analyze.remap_ae_table_loci(inf_ae_table, 
                                              saegus_to_tassel_loci)

In [63]:
expanded_inf_ae.to_hdf('expanded_infinite_allele_effects.hdf', 'exp')